# ERF and spectral analysis

Again, we start by importing the packages that we will use

In [ ]:
import mne
import numpy as np # This is a basic python package that includes tools for manipulating the data

In [ ]:
#data_path = 'C:\\Users\\yhare\\mne_data\\MNE-brainstorm-data\\bst_raw\\'
data_path = '/home/hyruuk/mne_data/MNE-brainstorm-data/bst_raw/'
fname = 'clean_dataset_epochs.fif.gz'
epochs = mne.read_epochs(data_path + fname, preload=True); # Here we load the cleaned data

### Plotting evoked data
Now, we can try to see if there is a difference between the evoked activity in our two conditions.

In [ ]:
evoked = epochs.average() # The function average() creates an Evoked MNE object, which is the average of all the epochs.


evoked.plot(); # First, we can make a "butterfly" view of the evoked activity (across both conditions)

We clearly see a response that is phase-locked to the stimulus. Let's look at the topographic distribution of this evoked activity.

In [ ]:
evoked.plot_topomap();

Look at the topography at 33ms. We can visually identify at least two generators. Are they somehow link to our experimental conditions ? Let's display them separately to confirm !

In [ ]:
epochs['Right'].average().plot_topomap(title='Right');
epochs['Left'].average().plot_topomap(title='Left');

Looks like it ! That makes sense because we know that a stimulation of the right hand will induce activity in the left sensorimotor cortices, and conversely.

Now, we could also plot the Event-Related Field (ERF), to compare the two conditions across time.

In [ ]:
# First, we have to build a dict containing the two conditions in separate entries
conditions = ["Right", "Left"]
evoked_dict = dict()

# Now we run a loop that will collect and average epochs separately for each condition
for condition in conditions:
    evoked_dict[condition] = epochs[condition].average().filter(l_freq=None, h_freq=30, method='iir') # Note that we apply a
                        # low-pass filter to the data (after averaging). This is a common practice to visualize data efficiently.
                                                                                
print(evoked_dict)

We can now plot an ERF of each condition !

In [ ]:
pick = epochs.ch_names.index('MLC21-4408') # I choose this sensor since it is just above sensorimotor areas.
colors = dict(Right='green', Left='red')
mne.viz.plot_compare_evokeds(evoked_dict, picks=pick, colors=colors);

In [ ]:
# Let's do the same for the same channel but in the other hemisphere
pick = epochs.ch_names.index('MRC21-4408')
colors = dict(Right='green', Left='red')

mne.viz.plot_compare_evokeds(evoked_dict, picks=pick, colors=colors);

The activity looked like it is reversed ! Again, this is exactly what we expected.

### Plotting spectral (PSD) data
Now that we did a quick overlook on the evoked activity, we should also perform some kind of spectral analysis.
Quantifying Power Spectrum Densities (PSD) in various frequency bands can inform us about the power distribution of cannonical oscillations. Alternatively, by using Time-Frequency Maps (TF maps) we can distinguish between evoked (phase-locked to the stimuli) and induced (not phase-locked) activity, which may behave differently.

In [ ]:
epochs.plot_psd(fmax=200); # First, let's have a look to the whole power spectrum. This is not very informative in our case 
                           # but it can help you to understand how your signal is structured (1/f, cut-off frequency etc...)

In [ ]:
epochs['Right'].plot_psd_topomap(); # You can also plot the topographies of frequency bands.
epochs['Left'].plot_psd_topomap();  # Let's compare both conditions !

Meh, we don't see anything really convincing... Maybe a more lateralised activity in the 'Right' condition ?
Let's explore the TF representation.

In [ ]:
freqs = np.logspace(*np.log10([6, 35]), num=12) # Define the frequency range (i.e. frequential definition)
n_cycles = freqs / 4.  # Different number of cycle per frequency (i.e. temporal definition)

power, itc = mne.time_frequency.tfr_morlet(epochs, freqs=freqs, n_cycles=n_cycles, use_fft=True, return_itc=True, decim=3, n_jobs=1)

power.plot_topo(baseline=baseline, mode='logratio', title='Average power', picks=None); # Plot the TF maps for each sensor

Since we want to compare Left and Right stimulations, we should build a TF map for each condition.

In [ ]:
power_right, itc_right = mne.time_frequency.tfr_morlet(epochs['Right'], freqs=freqs, n_cycles=n_cycles, use_fft=True, return_itc=True, decim=3, n_jobs=1)
power_left, itc_left = mne.time_frequency.tfr_morlet(epochs['Left'], freqs=freqs, n_cycles=n_cycles, use_fft=True, return_itc=True, decim=3, n_jobs=1)
# Note that here we only selected epochs from the Left or Right conditions.

In [ ]:
pick = power.ch_names.index('MLC21-4408')
power_right.plot([pick], baseline=baseline, mode='logratio', title='Right ' + power.ch_names[pick]);
power_left.plot([pick], baseline=baseline, mode='logratio', title='Left ' + power.ch_names[pick]);

In [ ]:
pick = power.ch_names.index('MRC21-4408')
power_right.plot([pick], baseline=baseline, mode='logratio', title='Right ' + power.ch_names[pick]);
power_left.plot([pick], baseline=baseline, mode='logratio', title='Left ' + power.ch_names[pick]);